In [1]:
import model
import torch
import os
from tkinter import *
from PIL import Image, ImageTk

chiros = model.Chiros()
chiros.load_state_dict(torch.load('chiros.pth'))
chiros.eval()
chiros.cuda()

class_names = ["pronation", "supination"]


import cv2
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Width of the frames in the video stream.
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  # Height of the frames in the video stream.
cap.set(cv2.CAP_PROP_FPS, 60)  # Frame rate.

# get image until q is pressed and feed through the model, add overlay and display

import numpy as np
import time
mean=torch.Tensor([0.485, 0.456, 0.406]).cuda()
std=torch.Tensor([0.229, 0.224, 0.225]).cuda()

imgtk = None
window = Tk()

image_canvas = Canvas(window, width=640, height=480)
image_canvas.pack()

# Create buttons
for i in range(len(class_names)):
    class_name = class_names[i]
    button = Button(window, text=class_name, command=lambda class_name=class_name: capture_and_save(os.path.join("images", class_name)))
    button.pack()

def normalize(x):
    return (x - mean) / std    

def capture_and_save(class_dir):
    global out_frame
    img_name = os.path.join(class_dir, "{}.jpg".format(len(os.listdir(class_dir))))
    cv2.imwrite(img_name, out_frame)

def update_image():
    global imgtk
    global out_frame
    ret, frame = cap.read()
    if not ret:
        return None
    tensor_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    out_frame = frame.copy()
    tensor_frame = torch.tensor(tensor_frame).float() / 255
    tensor_frame = normalize(tensor_frame.cuda()).permute(2, 0, 1).unsqueeze(0)
    pred = chiros(tensor_frame).softmax(1).cpu().detach().numpy()[0]
    
    # add overlay bar chart by drawing a rectangle
    # and filling it with a color that is proportional to the prediction
    # for each class
    
    for i, p in enumerate(pred):
        cv2.rectangle(frame, (0, i * 20), (int(p * 640), (i + 1) * 20), (0, 255, 0), -1)

    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    imgtk = ImageTk.PhotoImage(image=img)
    image_canvas.create_image(0, 0, anchor=NW, image=imgtk)

    window.after(10, update_image)

update_image()
# Start the main loop
window.mainloop()